# Analyze Forced Photometry in ComCam Data

In [ ]:
# %pip install lsdb dask nested-dask 

In [ ]:
import lsdb
lsdb.__version__

In [ ]:
import pandas as pd
pd.__version__

In [ ]:
from pathlib import Path

release = 'w_2025_07'
hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats/") / release
# list dir
print(list(map(str, hats_path.iterdir())))

comcam_obj = hats_path / "object"
comcam_src = hats_path / "forcedSource"

## Start Dask client

In [ ]:
from dask.distributed import Client

# Start with a small client
client = Client(n_workers=1, memory_limit="16GB", threads_per_worker=1)
client

# Load GAIA Data Around the COMCAM Field

In [ ]:
from upath import UPath


catalogs_dir = UPath("https://data.lsdb.io/hats")

# Gaia
gaia_path = catalogs_dir / "gaia_dr3" / "gaia"

# ZTF
ztf_object_path = catalogs_dir / "ztf_dr14" / "ztf_object"
ztf_source_path = catalogs_dir / "ztf_dr14" / "ztf_source"
ztf_object_margin_path = catalogs_dir / "ztf_dr14" / "ztf_object_10arcs"

# Define a 0.7 degree cone region of interest
# This covers the so called `Rubin_SV_95_-25` field, one of Data Preview 1 fields
cone_search = ConeSearch(ra=95, dec=-25, radius_arcsec=0.7 * 3600)

# Load lite version of Gaia DR3 (RA and DEC data only)
gaia_lite = lsdb.read_hats(gaia_path, columns=["ra", "dec"], search_filter=cone_search)

# Load all Gaia data
hats_gaia = hats.read_hats(gaia_path)
gaia = lsdb.read_hats(gaia_path, columns=hats_gaia.schema.names)

## Loading & Nesting Forced Sources

In [ ]:
# Load the Forced Source + MJD Table
from lsdb import read_hats


#BRIGHTEST_R_MAG = 21.5


obj = read_hats(
    comcam_obj,
    columns=["objectId", "coord_ra", "coord_dec"],
    #filters=[("r_psfMag", ">", BRIGHTEST_R_MAG)],
)
src_flat = read_hats(
    comcam_src,
    columns=[
        "objectId", 
        # "coord_ra", "coord_dec",
        "band",
        "midpointMjdTai",
        "psfFlux", "psfFluxErr", "psfFlux_flag",
        "psfMag", "psfMagErr",
        "pixelFlags_suspect", "pixelFlags_saturated", "pixelFlags_cr", "pixelFlags_bad",
        "forcedSourceId",
        "detector",
        "visit",
    ],
)
src_nested = obj.join_nested(
    src_flat,
    nested_column_name="lc",
    left_on="objectId",
    right_on="objectId",
)
src_nested